# 버스데이터를 기반으로 인구유동성 데이터 추출하기 

#### 필요 파일 및 패키지 불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
bustatonf = pd.read_csv('20.고양시_버스정류소.csv')
bustat = pd.read_csv('21.버스_정류장별_승하차_정보.csv')

## 데이터 설명 
 ### 20.고양시_버스정류소
 - 현재 기준의 고양시 버스 정류소, 마을버스 정류소 위치
- STATION_NM    버스정류장 명
- STATION_ID    버스정류장 고유 ID
- X    버스정류장의 위도
- Y    버스정류장의 경도

 ### 21.버스_정류장별_승하차_정보
 - 최근 1년치 고양시 버스 정류장별 승하차 정보
- STATION_ID    버스정류장 고유 ID
- STATION_NM    버스정류장 명
- GETON_CNT    버스정류장 승차인원 수

### 두 파일을 병합하기 
- STATION_ID: 정류장ID
- STATION_NM: 정류장 이름
- GETON_CNT: 승하차인원수
- X: 경도
- Y: 위도

In [ ]:
# merge 이용
bus = bustat.merge(bustatonf, on=['STATION_NM','STATION_ID'])
bus

#2823개

,STATION_ID,STATION_NM,GETON_CNT,X,Y
0,218001046,화정역3호선,229091,126.832410,37.635347
1,218001083,원당역,219186,126.843885,37.653079
2,218000974,화정역3호선,197084,126.832827,37.634486
3,218000974,화정역3호선,5335,126.832827,37.634486
4,219000193,대화역,191696,126.747539,37.676390
...,...,...,...,...,...
2818,219000882,환경사업소,0,126.726069,37.659134
2819,219000968,장항굴다리,0,126.761935,37.647081
2820,218000827,새벽교회,0,126.858807,37.721591
2821,218000888,남양유업,0,126.852764,37.666689


## groupby를 통해 동명의 정류장끼리 묶는다
 - 위치가 조금씩 다르지만 같은 한글이름을 사용하는 정류장들을 묶음

In [ ]:
bus_grouped = bus.groupby(['STATION_NM','STATION_ID','GETON_CNT'],as_index=False).sum()
bus_grouped

,STATION_NM,STATION_ID,GETON_CNT,X,Y
0,(구)효자동주민센터,218000258,149,126.919747,37.652634
1,(구)효자동주민센터,218000258,195,126.919747,37.652634
2,(구)효자동주민센터,218000265,19,126.919892,37.652603
3,(구)효자동주민센터,218000265,27,126.919892,37.652603
4,16단지.삼송초교,218001182,9686,126.882791,37.653627
...,...,...,...,...,...
2818,흰돌마을(중),219000453,6900,126.781373,37.648172
2819,흰돌마을(중),219000453,7817,126.781373,37.648172
2820,흰돌마을.서안아파트,219000161,934,126.785637,37.641313
2821,흰돌마을.서안아파트,219000161,10467,126.785637,37.641313


In [ ]:
# 재정의
bus2 = bus_grouped

## 동명의 정류장 중 경위도값이 같은 정류장끼리는 GETON_CNT를 더해준다 (groupby)

In [ ]:
# 위치가 같은 정류소 중복을 방지
bus3 = bus2.groupby(['STATION_NM','STATION_ID','X','Y'], as_index=False).sum()
bus3

,STATION_NM,STATION_ID,X,Y,GETON_CNT
0,(구)효자동주민센터,218000258,126.919747,37.652634,344
1,(구)효자동주민센터,218000265,126.919892,37.652603,46
2,16단지.삼송초교,218001182,126.882791,37.653627,9686
3,16단지.삼송초교,218001187,126.883295,37.653999,1126
4,1군단앞,218000498,126.911514,37.716309,21
...,...,...,...,...,...
2076,희망원,219000229,126.821292,37.702977,1248
2077,흰돌5단지.서안아파트,219000213,126.785729,37.640872,14603
2078,흰돌마을(중),219000453,126.781373,37.648172,14717
2079,흰돌마을.서안아파트,219000161,126.785637,37.641313,11401


In [ ]:
bus3.head(10)

,STATION_NM,STATION_ID,X,Y,GETON_CNT
0,(구)효자동주민센터,218000258,126.919747,37.652634,344
1,(구)효자동주민센터,218000265,126.919892,37.652603,46
2,16단지.삼송초교,218001182,126.882791,37.653627,9686
3,16단지.삼송초교,218001187,126.883295,37.653999,1126
4,1군단앞,218000498,126.911514,37.716309,21
5,1군단앞,218000500,126.912488,37.717179,321
6,2단지관리사무실앞,218000743,126.836772,37.646522,303
7,2단지라이프아파트앞,218000932,126.837170,37.645662,24
8,2단지라이프아파트앞,218001009,126.837020,37.645986,14376
9,5단지관리사무실앞,218000740,126.836483,37.642546,7119


## 상위 몇개만 뽑아보면 버스승하차 횟수가 많은 지역을 알 수 있을 것이다

In [ ]:
bus_a = bus3.sort_values(by='GETON_CNT', ascending=False)
bus_a

,STATION_NM,STATION_ID,X,Y,GETON_CNT
384,대화역,219000193,126.747539,37.676390,361686
2023,화정역3호선,218001046,126.832410,37.635347,229091
1340,원당역,218001083,126.843885,37.653079,219186
2022,화정역3호선,218000974,126.832827,37.634486,202419
385,대화역,219000366,126.747416,37.675989,199238
...,...,...,...,...,...
1325,원골마을회관,218000597,126.864203,37.581361,0
888,새벽교회,218000827,126.858807,37.721591,0
398,덕은동,218000594,126.860548,37.581514,0
56,가좌동성당,219001111,126.726562,37.697732,0


In [ ]:
# 전체 버스 정류장 평균 승하차인원이 4700여명 정도 된다
bus_a.describe() 

,STATION_ID,X,Y,GETON_CNT
count,2.081000e+03,2081.000000,2081.000000,2081.000000
mean,2.185258e+08,126.819148,37.666634,4688.834695
std,1.033896e+06,0.053995,0.032485,16699.517366
min,2.180000e+08,126.679997,37.581102,0.000000
25%,2.180008e+08,126.777363,37.644999,70.000000
50%,2.180013e+08,126.823145,37.668142,573.000000
75%,2.190006e+08,126.859858,37.691194,3677.000000
max,2.415011e+08,126.962809,37.747089,361686.000000


## 승하차별 상위 n개의 버스데이터를 만들어 살펴보자 
 - csv파일로 만들어 저장한 뒤, qgis프로그램을 이용하여 시각화할 수 있음

In [ ]:
# 이용이 많은 버스정류장 상위 30개를 살펴볼 수 있다
bus_a_top30 = bus_a.head(30)

# 이용이 많은 버스정류장 상위 50개를 살펴볼 수 있다
bus_a_top50 = bus_a.head(50)

# 이용이 많은 버스정류장 상위 100개를 살펴볼 수 있다
bus_a_top100 = bus_a.head(100)

In [ ]:
# csv파일로 만들어 추후 qgis 시각화에 이용
bus_a_top30.to_csv('bus_a_top30.csv')
bus_a_top50.to_csv('bus_a_top50.csv')
bus_a_top100.to_csv('bus_a_top100.csv')

In [ ]:
# 재정의
bus_a_top100 = bus_a_top100.copy()
bus_a_top100

,STATION_NM,STATION_ID,X,Y,GETON_CNT
384,대화역,219000193,126.747539,37.676390,361686
2023,화정역3호선,218001046,126.832410,37.635347,229091
1340,원당역,218001083,126.843885,37.653079,219186
2022,화정역3호선,218000974,126.832827,37.634486,202419
385,대화역,219000366,126.747416,37.675989,199238
...,...,...,...,...,...
899,샘터마을2단지.신능중학교,218000118,126.833738,37.621644,19139
1916,행신역,218001019,126.835049,37.612941,18961
1087,숲속마을4.5단지,219000460,126.800155,37.669089,18927
1745,탄현효성주은아파트,219000315,126.768340,37.690707,18923


- 중복되는 정류장 (= 상/하행이 있는 정류장)과 중복되지 않는 정류장 분류
- 중복되는 정류장 -> 양방향 유동인구가 많다는 것을 의미 -->  합쳐진 데이터로 분석


In [ ]:
# 여기서는 중복되지 않는 정류장 이름들을 살펴본다

bus_a_top100_no_dup = bus_a_top100.drop_duplicates(["STATION_NM"], keep=False)
bus_a_top100_no_dup.info()

# # 총59곳 --> 중복되는 것의 짝이 두개씩이면 중복되지 않는 것도 짝수 값이 나올텐데
# 홀수값 --> 정류장이름이 중복되는 것이 세개씩 있는 것도 있을 것이라고 추측가능함.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 1340 to 442
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   STATION_NM  59 non-null     object 
 1   STATION_ID  59 non-null     int64  
 2   X           59 non-null     float64
 3   Y           59 non-null     float64
 4   GETON_CNT   59 non-null     int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 2.8+ KB


In [ ]:
# 중복되지 않는 정류장
bus_a_top100_no_dup

,STATION_NM,STATION_ID,X,Y,GETON_CNT
1340,원당역,218001083,126.843885,37.653079,219186
855,삼송역8번출구,218001200,126.896880,37.653835,102780
853,삼송역5번출구,218000284,126.893512,37.653499,82318
2020,화정역.롯데마트,218000019,126.830748,37.632918,66042
851,삼송역3번출구,218001280,126.895430,37.652687,58272
854,삼송역6번출구,218000627,126.896329,37.653501,55333
852,삼송역4번출구,218000090,126.893807,37.653250,55069
1367,원흥역3번출구,218000696,126.873272,37.650091,50388
2013,화정1동행정복지센터,218000102,126.834066,37.646016,46277
1947,현대백화점,219000625,126.752013,37.668475,46162


In [ ]:
# 중복되는 정류소 뽑기

bus_top_data = pd.concat([bus_a_top100, bus_a_top100_no_dup, bus_a_top100_no_dup]).drop_duplicates(keep=False)
bus_top_data

# "리스쇼핑.원당시장"은 값이 3개다
# (지역 특성상) 로타리라서 정류장이 많음 (총 4개)
# 이 곳은 양방향 유동인구가 많다고 할 수 있겠다

,STATION_NM,STATION_ID,X,Y,GETON_CNT
384,대화역,219000193,126.747539,37.676390,361686
2023,화정역3호선,218001046,126.832410,37.635347,229091
2022,화정역3호선,218000974,126.832827,37.634486,202419
385,대화역,219000366,126.747416,37.675989,199238
534,마두역(중),219000370,126.777480,37.652390,162142
1478,일산동구청(중),219000356,126.774287,37.657816,156731
1590,주엽역(중),219000189,126.761634,37.670067,154395
1477,일산동구청(중),219000191,126.774728,37.657438,151423
535,마두역(중),219000383,126.778044,37.651717,148257
1591,주엽역(중),219000363,126.760893,37.670280,119326


In [ ]:
# 총41곳
bus_top_data['STATION_NM'].count()

41

In [ ]:
# 짝이 다 맞는지 확인하기
bus_top_data.groupby('STATION_NM').count()

,STATION_ID,X,Y,GETON_CNT
STATION_NM,,,,
강선마을(중),2,2,2,2
고양경찰서(중),2,2,2,2
고양동시장,2,2,2,2
관산동.벽제시장,2,2,2,2
대화역,2,2,2,2
리스쇼핑.원당시장,3,3,3,3
마두1동행정복지센터,2,2,2,2
마두역(중),2,2,2,2
문촌마을(중),2,2,2,2
